<h1><center><b>NeutrinAI</b></center></h1>

# **Model description**

- Input Features
    - 6 features are used as input.
        - $\vec{f} \equiv $ $<$ $t$, $a$(uxiliary), $c$(harge), $x$, $y$, $z$ $>$
- Two RSBlocks
    - I have implemented `RSBlock` class. It consists of H (low level feature) layer, M (mapping) layer, FR (feature raising) layer, A (aggregation) layer and CR (channel raising) layer.
    - H, Low Level Features for Relation-Shape
        - $\rm \left(B_{atch}, N_{umber-of-pulses}, F_{eatures} \right) \rightarrow \left( B, N, K_{nearest-neighbors}, H_{low-level-features} \right)$
        - For each points $\vec{f_{0}}$ and its time-nearest points $\vec{f_{k}}$, evaluate low level feautres $\vec{h_{k, 0}} = \left< t_{k} - t_{0}, x_{k} - x_{0}, \rm{distance}, \cdots \right>$.
    - M, Mapping Layer
        - $\rm \left(B, N, K, F_{in} \right) \rightarrow \left( B, N, K, F_{out} \right)$
        - From $\rm F_{in}$ number of low-level features, generated $\rm F_{out}$ number of higher-level features.
        - Conv2d with $1 \times 1$ window was used.
    - FR, Feature Raising Layer
        - $\rm \left(B, N, F_{eatures} \right) \rightarrow \left( B, N, F_{FR} \right)$
        - For the feature vector of the point (not neighbor), $\vec{f_{0}}$, increase its features into higher level.
        - Conv1d with window length of 1 was used.
    - A, Aggregation Layer
        - So, now we have two tensors.
            - $FR_{n, f}$ with shape of $\rm \left( B, N, F_{FR} \right)$ from the point, and $M_{nkf}$ with shape of $\rm \left( B, N, K, F_{out} \right)$ from its neighbors.
            - $\rm F_{FR}$ and $\rm F_{out}$ must be same, 
        - Calculate tensor $T$ as component-wise product, $T_{n, k, f} = FR_{n, f} \cdot M_{n, k, f}$.
        - Aggregate features from its neighbors. $A_{n, f} = \rm{max}_{among ~ \it k} \it \left( T_{n, k, f} \right)$.
        - Now, we have $A_{n, f}$ tensor with shape of $\rm \left(B, N, F_{FR} \right)$
    - CR, Channel Raising Layer
        - $\rm \left(B, N, F_{FR} \right) \rightarrow \left( B, N, F_{CR} \right)$
        - Raise number of channels using 1D Conv with the window length of 1.
- GABlock with RNN
    - $\rm \left(B, N, F_{CR} \right) \rightarrow \left( B, 3 \right)$
    - Globally aggregate relation-shape pulse features to predict the angle.
    - 3 GRU layers were adopted here.
    - Attached one hidden Dense layer after the series of GRUs, and one output Dense layer for prediction.

# **Relation-Shape-RNN**

- Why RNN-based?
    - Light LSTM model showed some performance.
    - These may imply there could be some potential on RNN-based method. So I decided to bet my effort on it.
        - It is much easier to understand than GNN. Don't need other packages like `~_geometry` or `~_knn~`.
- Relation-Shape?
    - The key point was to train the `local shape` features. They gathered some nearest neighbors using knn, evaluated some local shape features like $d\vec{x}$ or euclidean distance, and applied cnn on those features.
- How to train Relation-Shape in RNN?
    - I've interpreted our dataset as 4-dimensional point cloud.
        - $x^{\mu} = \left< t, ~ x, ~ y, ~ z \right>$
    - My guess (and maybe the most important reason for the low performance of my model) was that the *nearest neighbor* in our data can be approximated as *time-nearest neighbor*.
    - So, I have implemented the relation-shape learning as 1D-Conv network, where the points are sorted in the time order.


# **Loss function**

I've implemented and used `VonMisesFisher3DLoss

# **Libraries**

In [ ]:
!mkdir ./Packages
!cp /neutrinai-packages/*.py ./Packages
!ls ./Packages/

DataIO.py  Layers.py  Losses.py  Metrics.py  Utils.py  __init__.py


In [ ]:
import os,random,time,gc,matplotlib as mpl,matplotlib.pyplot as plt, pandas as pd
from tqdm import tqdm
import numpy as np,pandas as pd,pyarrow.parquet as pq,pyarrow as pa,tensorflow as tf
from Packages.Layers import GABlockResRNN, RSBlock
from Packages.Losses import VonMisesFisher3DLoss
from Packages.Metrics import AngularDistScore, angular_dist_score
from Packages.Utils import GpuMemoryManagement
from typing import List

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# **Sensor geometry**

In [ ]:
# sensor_geometry
sensor_geometry_df = pd.read_csv("neutrinai-packages/sensor_geometry.csv")

# counts
doms_per_string = 60
string_num = 86

# index
outer_long_strings = np.concatenate([np.arange(0, 25), np.arange(27, 34), np.arange(37, 44), np.arange(46, 78)])
inner_long_strings = np.array([25, 26, 34, 35, 36, 44, 45])
inner_short_strings = np.array([78, 79, 80, 81, 82, 83, 84, 85])

# known specs
outer_xy_resolution = 125. / 2
inner_xy_resolution = 70. / 2
long_z_resolution = 17. / 2
short_z_resolution = 7. / 2

# evaluate error
sensor_x = sensor_geometry_df.x
sensor_y = sensor_geometry_df.y
sensor_z = sensor_geometry_df.z
sensor_r_err = np.ones(doms_per_string * string_num)
sensor_z_err = np.ones(doms_per_string * string_num)

# r-error
for string_id in outer_long_strings:
    sensor_r_err[string_id * doms_per_string:(string_id + 1) * doms_per_string] = outer_xy_resolution
for string_id in np.concatenate([inner_long_strings, inner_short_strings]):
    sensor_r_err[string_id * doms_per_string:(string_id + 1) * doms_per_string] = inner_xy_resolution

# z-error
for string_id in outer_long_strings:
    sensor_z_err[string_id * doms_per_string:(string_id + 1) * doms_per_string] = long_z_resolution
for string_id in np.concatenate([inner_long_strings, inner_short_strings]):
    for dom_id in range(doms_per_string):
        z = sensor_z[string_id * doms_per_string + dom_id]
        if (z < -156.) or (z > 95.5 and z < 191.5):
            sensor_z_err[string_id * doms_per_string + dom_id] = short_z_resolution
        else:
            sensor_z_err[string_id * doms_per_string + dom_id] = long_z_resolution
# register
sensor_geometry_df["r_err"] = sensor_r_err
sensor_geometry_df["z_err"] = sensor_z_err

In [ ]:
# detector constants
c_const = 0.299792458  # speed of light [m/ns]

x_min = sensor_x.min()
x_max = sensor_x.max()
y_min = sensor_y.min()
y_max = sensor_y.max()
z_min = sensor_z.min()
z_max = sensor_z.max()

detector_length = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2 + (z_max - z_min)**2)
t_valid_length = detector_length / c_const

print("t_valid_length: ", t_valid_length, " ns")

t_valid_length:  6199.700247193777  ns


# **Generator from parquet**

In [ ]:
def generator_from_parquet(
    meta_data_path,
    data_path_header,
    max_pulse_count: int=128,
    max_batch_id: int=9999,
):
    # type conversion for tf
    if type(meta_data_path) == bytes:
        meta_data_path = str(meta_data_path, 'utf-8')
    if type(data_path_header) == bytes:
        data_path_header = str(data_path_header, 'utf-8')
        
    print(meta_data_path)
    print(data_path_header)
    
    # read one batch
    meta_data = pq.ParquetFile(meta_data_path)
    for meta_batch in meta_data.iter_batches(batch_size=200000):
        # batch meta data
        batch_id = meta_batch["batch_id"][0].as_py()
        event_ids = meta_batch["event_id"].to_numpy()
        
        if batch_id > max_batch_id:
            print("Reached max_batch_id!")
            return
        
        # batch data
        data_batch = pq.read_table(data_path_header + f"{batch_id:d}.parquet")
        sensor_id = data_batch["sensor_id"].combine_chunks().to_numpy()
        time = data_batch["time"].combine_chunks().to_numpy()
        charge = data_batch["charge"].combine_chunks().to_numpy()
        auxiliary = data_batch["auxiliary"].combine_chunks().to_numpy(False)
        pulse_index = np.append(meta_batch["first_pulse_index"].to_numpy(), [data_batch.num_rows])
        
        # read each event
        for event_id, first_idx, last_idx in zip(event_ids, pulse_index[:-1], pulse_index[1:]):
            event_time = time[first_idx:last_idx]
            event_time = event_time - event_time.min()
            event_charge = charge[first_idx:last_idx]
            event_auxiliary = auxiliary[first_idx:last_idx]
            event_x = sensor_geometry_df.x[sensor_id[first_idx:last_idx]].values
            event_y = sensor_geometry_df.y[sensor_id[first_idx:last_idx]].values
            event_z = sensor_geometry_df.z[sensor_id[first_idx:last_idx]].values
            
            dtype = [
                ("time", "float16"),
                ("charge", "float16"),
                ("auxiliary", "float16"),
                ("x", "float16"),
                ("y", "float16"),
                ("z", "float16"),
                ("rank", "short")
            ]
            event_features = np.zeros(last_idx - first_idx, dtype)
            event_features["time"] = event_time
            event_features["charge"] = event_charge
            event_features["auxiliary"] = event_auxiliary
            event_features["x"] = event_x
            event_features["y"] = event_y
            event_features["z"] = event_z

            # point picker
            if len(event_x) > max_pulse_count:
                # find valid time window
                t_peak = event_features["time"][event_features["charge"].argmax()]
                t_valid_min = t_peak - t_valid_length
                t_valid_max = t_peak + t_valid_length
                
                # rank
                t_valid = (event_features["time"] > t_valid_min) * (event_features["time"] < t_valid_max)
                event_features["rank"] = 2 * (1 - event_features["auxiliary"]) + (t_valid)
                
                # sort by rank and charge
                event_features = np.sort(event_features, order=["rank", "charge"])
                
                # pick-up from backward
                event_features = event_features[-max_pulse_count:]
                
                # resort by time
                event_features = np.sort(event_features, order="time")
            
            pulse_count = min(len(event_x), max_pulse_count)
            
            # yield
            features = np.zeros((max_pulse_count, 6), dtype="float32")
            features[:pulse_count, 0] = (event_features["time"] - event_features["time"].min()).astype("float32") / 1000.
            features[:pulse_count, 1] = event_features["charge"].astype("float32") / 300.
            features[:pulse_count, 2] = event_features["auxiliary"].astype("float32") * 1.
            features[:pulse_count, 3] = event_features["x"].astype("float32") / 600.
            features[:pulse_count, 4] = event_features["y"].astype("float32") / 600.
            features[:pulse_count, 5] = event_features["z"].astype("float32") / 600.

            features[:, :][features[:, 1] == 0.0] = 0.0  # for masking layer
            
            yield event_id, features
        
        # memory management
        del auxiliary, charge, time, sensor_id, data_batch, pulse_index, event_id
        _=gc.collect()

# **Loading model**

In [ ]:
model_dir = "/neutrinai-packages/"

model_da_name = "best_model"
model_da = tf.keras.models.load_model(
    model_dir + model_da_name + ".h5",
    custom_objects={
        "VonMisesFisher3DLoss": VonMisesFisher3DLoss,
        "AngularDistScore": AngularDistScore,
        "RSBlock": RSBlock,
        "GABlockResRNN": GABlockResRNN,
    },
)
model_da.summary()

Model: "RSGRUDA_s3260114_s595_s756_s639_s585_s403_s158_s263_s756"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 6)]     0           []                               
                                                                                                  
 Masking (Masking)              (None, 128, 6)       0           ['input_1[0][0]']                
                                                                                                  
 rs_block (RSBlock)             ((None, 128, 6),     6820        ['Masking[0][0]']                
                                 (None, 128, 93))                                                 
                                                                                                  
 rs_block_1 (RSBlock)           ((None, 128

# **Projections in test dataset**

In [ ]:
max_batch_id = 99999
batch_size = 128

if max_batch_id < 9999:
    print("Starting")
    split_name = "train"
else:
    split_name = "test"

home_dir = "/wowdao-self-supervised-learning-dataset/"
meta_data_path = home_dir + split_name + '_meta.parquet'
data_dir = home_dir + split_name + "/"
data_path_header = data_dir + "batch_"

max_pulse_count = 128
max_pulse_count_lite = 80

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
test_da_ds = tf.data.Dataset.from_generator(
    generator_from_parquet,
    args=[meta_data_path, data_path_header, max_pulse_count, max_batch_id],
    output_types=(tf.int32, tf.float32),
    output_shapes=((None), (max_pulse_count, 6)),
)
test_da_ds = test_da_ds.batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
with open('pred_da.csv', 'w') as pred_csv:
    pred_csv.write('event_id,pred_x,pred_y,pred_z\n')

with open('pred_lite.csv', 'w') as pred_csv:
    pred_csv.write('event_id,pred_x,pred_y,pred_z,pulse\n')

In [ ]:
%%time

steps_per_batch = np.ceil(200000 // batch_size)
print(f"One batch has {int(steps_per_batch):d} steps")

step = 0
for batch_data in test_da_ds:
    step += 1
    print(f"{step:7d}", end="")
    if (step % 20 == 0) or (step % steps_per_batch == 0):
        print()
        
    batch_event_id, batch_features = batch_data
#     batch_features[:, :, :][batch_features[:, :, 1] == 0.0] = 0.0  # for masking
    
    test_pred = model_da.predict_on_batch(batch_features)
    
    with open('pred_da.csv', 'a') as pred_csv:
        for event_id, pred_x, pred_y, pred_z in zip(batch_event_id, test_pred[:, 0], test_pred[:, 1], test_pred[:, 2]):
            pred_csv.write(f"{event_id:d},{pred_x:f},{pred_y:f},{pred_z:f}\n")
    
    del batch_event_id, batch_features, test_pred

One batch has 1562 steps
/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet
/kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_
      1CPU times: user 8.85 s, sys: 253 ms, total: 9.1 s
Wall time: 9.09 s


# **Memory management**

Discard models from GPU and RAM

In [ ]:
del model_da
tf.keras.backend.clear_session()
print("gc collect: ", gc.collect())

gc collect:  62459


# **Conversion to angle**

In [ ]:
# read predictions
pred_da = pd.read_csv("pred_da.csv")

event_id = pred_da.event_id.values

pred_da_x = pred_da.pred_x.values
pred_da_y = pred_da.pred_y.values
pred_da_z = pred_da.pred_z.values

In [ ]:
test_pred = np.zeros((len(pred_da_x), 3))

test_pred[:, 0] = pred_da_x
test_pred[:, 1] = pred_da_y
test_pred[:, 2] = pred_da_z

# convert to angle
kappa_pred = np.linalg.norm(test_pred, axis=1)
vec_x_pred = test_pred[:, 0] / kappa_pred
vec_y_pred = test_pred[:, 1] / kappa_pred
vec_z_pred = test_pred[:, 2] / kappa_pred
az_pred = np.arctan2(vec_y_pred, vec_x_pred)
az_pred = np.where(az_pred < 0, az_pred + 2*np.pi, az_pred)
zen_pred = np.arccos(vec_z_pred)

# **Results**

In [ ]:
with open('results.csv', 'w') as results:
    results.write('event_id,azimuth,zenith\n')
    
with open('results.csv', 'a') as results:
    for event, azimuth, zenith in zip(event_id, az_pred, zen_pred):
        results.write(f"{event:d},{azimuth:f},{zenith:f}\n")